In [73]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [74]:
import os

spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (225 kB/s)
Reading package lists... Done


In [75]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [76]:
# Read in the healthcare-dataset-stroke-data.csv via AWS into Spark DataFrame
from pyspark import SparkFiles
url = "https://project4-06052023.s3.us-east-2.amazonaws.com/healthcare-dataset-stroke-data.csv"
spark.sparkContext.addFile(url)
stroke_data = spark.read.csv(SparkFiles.get("healthcare-dataset-stroke-data.csv"), sep=",", header=True, inferSchema=True)
stroke_data.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [77]:
# from google.colab import drive
# drive.mount('/content/drive')

# **Preprocessing**

In [78]:
# Print Spark dataframe schema (Note: all schema except 'bmi' inferred correctly)
stroke_data.printSchema

<bound method DataFrame.printSchema of DataFrame[id: int, gender: string, age: double, hypertension: int, heart_disease: int, ever_married: string, work_type: string, Residence_type: string, avg_glucose_level: double, bmi: string, smoking_status: string, stroke: int]>

In [79]:
# Convert Spark dataframe to Pandas df
stroke_data_df = stroke_data.toPandas()

In [80]:
# Drop the non-beneficial ID column.
stroke_df = stroke_data_df.drop(columns={'id'})
stroke_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int32  
 3   heart_disease      5110 non-null   int32  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   object 
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int32  
dtypes: float64(2), int32(3), object(6)
memory usage: 379.4+ KB


In [81]:
# Convert 'bmi' to float (Note: 'coerce' converts 'N/A' values to NaN)
stroke_df['bmi'] = pd.to_numeric(stroke_df['bmi'], errors ='coerce')
stroke_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int32  
 3   heart_disease      5110 non-null   int32  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int32  
dtypes: float64(3), int32(3), object(5)
memory usage: 379.4+ KB


In [82]:
# Drop rows containing NaN
stroke_df = stroke_df.dropna()
stroke_df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [83]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_stroke_data = pd.get_dummies(stroke_df)
encoded_stroke_data

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5,81.0,0,0,186.21,29.0,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,13.0,0,0,103.08,18.6,0,1,0,0,1,...,0,0,0,1,1,0,1,0,0,0
5106,81.0,0,0,125.20,40.0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0
5107,35.0,0,0,82.99,30.6,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5108,51.0,0,0,166.29,25.6,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0


In [84]:
# Split our preprocessed data into our features and target arrays
y = encoded_stroke_data["stroke"]
X = encoded_stroke_data.drop(["stroke"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [85]:
# Check the balance of our target values
y.value_counts()

0    4700
1     209
Name: stroke, dtype: int64

In [86]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# **Round 1: Initial Testing**
Preliminary testing of a variety of machine learning models used for predicting outcomes of our labeled categorical data. These first attempts utilize our encoded data with no additional modifications.

### **Logistic Regression Model 1**
Attempt 1 using orignial data.

In [87]:
# Fit the model
# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(max_iter=200)

# Fit the model using training data
model = logistic_regression_model.fit(X_train_scaled, y_train)

In [88]:
# Make a prediction using the testing data
LR_pred = logistic_regression_model.predict(X_test_scaled)

In [89]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, LR_pred)

0.5

In [90]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, LR_pred)
print(matrix)

[[1179    0]
 [  49    0]]


In [91]:
# Print the classification report for the model
report = classification_report(y_test, LR_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1179
           1       0.00      0.00      0.00        49

    accuracy                           0.96      1228
   macro avg       0.48      0.50      0.49      1228
weighted avg       0.92      0.96      0.94      1228



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Logistic Regression Model**: Due to our unbalanced data set this model was unable to identify our positive targets.

### **Neural Network Model 1**
Attempt 1 using original data.

In [92]:
# Define the model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 9)                 198       
                                                                 
 dense_9 (Dense)             (None, 9)                 90        
                                                                 
 dense_10 (Dense)            (None, 9)                 90        
                                                                 
 dense_11 (Dense)            (None, 1)                 10        
                                                                 
Total params: 388
Trainable params: 388
Non-trainable params: 0
_________________________________________________________________


In [93]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [94]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
116/116 [==============================] - 1s 2ms/step - loss: 0.5439 - accuracy: 0.7802
Epoch 2/50
116/116 [==============================] - 0s 3ms/step - loss: 0.2052 - accuracy: 0.9565
Epoch 3/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1682 - accuracy: 0.9565
Epoch 4/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1604 - accuracy: 0.9565
Epoch 5/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1559 - accuracy: 0.9565
Epoch 6/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1526 - accuracy: 0.9565
Epoch 7/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1503 - accuracy: 0.9565
Epoch 8/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1482 - accuracy: 0.9565
Epoch 9/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1466 - accuracy: 0.9565
Epoch 10/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1447 - accuracy: 0.9565

In [95]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 [==============================] - 0s 1ms/step - loss: 0.1545 - accuracy: 0.9560
Loss: 0.15450340509414673, Accuracy: 0.9560260772705078


In [96]:
# Export our model to HDF5 file
# nn.save('Models/stroke_model_NN_1.h5')

In [97]:
# Compare predictions to actual outcomes
y_values = nn.predict(X_test_scaled)
y_predict = y_values.round()

df = pd.DataFrame(list(zip(y_test, y_predict)), columns =['true_value', 'predictions'])
df = df.round()
df

39/39 [==============================] - 0s 2ms/step


,true_value,predictions
0,0,[0.0]
1,0,[0.0]
2,0,[0.0]
3,0,[0.0]
4,0,[0.0]
...,...,...
1223,0,[0.0]
1224,0,[0.0]
1225,0,[0.0]
1226,0,[0.0]


In [98]:
# Print classification report
nn_report = classification_report(y_test, y_predict)
print(nn_report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1179
           1       0.00      0.00      0.00        49

    accuracy                           0.96      1228
   macro avg       0.48      0.50      0.49      1228
weighted avg       0.92      0.96      0.94      1228



**Neural Network Model:** This model failed to recognize any of our stroke patients.

### **K Nearest Neighbors 1**
Attempt 1 using original data

In [99]:
# Instantiate the model with k = 3 neighbors
model = KNeighborsClassifier(n_neighbors=3)

In [100]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [101]:
# Create predictions
KN_pred = model.predict(X_test_scaled)

In [102]:
# Print confusion matrix
confusion_matrix(KN_pred,y_test)

array([[1168,   48],
       [  11,    1]])

In [103]:
# Print classification report
print(classification_report(KN_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1216
           1       0.02      0.08      0.03        12

    accuracy                           0.95      1228
   macro avg       0.51      0.52      0.50      1228
weighted avg       0.98      0.95      0.97      1228



**K Nearest Neighbors:** This model also struggled to recognize our positive targets due to our unbalanced dataset, however it did attempt to classify a handful of data points as positive and correctly identified 4 unlike the logistic regression model.

### **Random Forest 1**
Attempt 1 using orignial data

In [104]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500)

In [105]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [106]:
# Making predictions using the testing data
rf_pred = rf_model.predict(X_test_scaled)

In [107]:
# Print confusion matrix
confusion_matrix(rf_pred,y_test)

array([[1177,   49],
       [   2,    0]])

In [108]:
# Print classification report
print(classification_report(rf_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1226
           1       0.00      0.00      0.00         2

    accuracy                           0.96      1228
   macro avg       0.50      0.48      0.49      1228
weighted avg       1.00      0.96      0.98      1228



**Random Forest:** This model also struggled due to the unbalanced data, performing slighly worse than the K Nearest Neighbors model.

# **Optimization**

## **Round 2: Resampling**
In our intial round of testing we found our unbalanced dataset, which contained only `4.26%` positive cases, was insufficient to train an accurate machine learning model. The second round of testing will utilize the `RandomOverSampler` from `imblearn` correct for this imbalance.


### **Resample Data with RandomOverSampler**

In [109]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
# Instantiate the random oversampler model
ros = RandomOverSampler()

# Fit the original training data to the random_oversampler model
X_R, y_R = ros.fit_resample(X_train, y_train)

In [111]:
# Count the distinct values of the resampled labels data
y_R.value_counts()

0    3521
1    3521
Name: stroke, dtype: int64

In [112]:
X_train_scaled_R = X_scaler.transform(X_R)

### **Neural Network Model 2**
Attempt 2 using RandomOverSampler and the same number of neurons/layers as the initial model.

In [113]:
# Neural network model with RandomOverSampler
# Define the model
number_input_features = len(X_train_scaled_R[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 9)                 198       
                                                                 
 dense_13 (Dense)            (None, 9)                 90        
                                                                 
 dense_14 (Dense)            (None, 9)                 90        
                                                                 
 dense_15 (Dense)            (None, 1)                 10        
                                                                 
Total params: 388
Trainable params: 388
Non-trainable params: 0
_________________________________________________________________


In [114]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [115]:
# Train the model
fit_model = nn.fit(X_train_scaled_R,y_R,epochs=50)

Epoch 1/50
221/221 [==============================] - 2s 4ms/step - loss: 0.6433 - accuracy: 0.6460
Epoch 2/50
221/221 [==============================] - 1s 6ms/step - loss: 0.5370 - accuracy: 0.7381
Epoch 3/50
221/221 [==============================] - 1s 6ms/step - loss: 0.4657 - accuracy: 0.7843
Epoch 4/50
221/221 [==============================] - 2s 7ms/step - loss: 0.4351 - accuracy: 0.7966
Epoch 5/50
221/221 [==============================] - 1s 4ms/step - loss: 0.4162 - accuracy: 0.8082
Epoch 6/50
221/221 [==============================] - 1s 4ms/step - loss: 0.4047 - accuracy: 0.8134
Epoch 7/50
221/221 [==============================] - 1s 4ms/step - loss: 0.3938 - accuracy: 0.8177
Epoch 8/50
221/221 [==============================] - 1s 5ms/step - loss: 0.3838 - accuracy: 0.8231
Epoch 9/50
221/221 [==============================] - 1s 4ms/step - loss: 0.3749 - accuracy: 0.8340
Epoch 10/50
221/221 [==============================] - 1s 5ms/step - loss: 0.3667 - accuracy: 0.8354

In [116]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.6605 - accuracy: 0.7915 - 345ms/epoch - 9ms/step
Loss: 0.6604593992233276, Accuracy: 0.791530966758728


In [117]:
# Compare predictions to actual outcomes
y_values2 = nn.predict(X_test_scaled)
y_predict2 = y_values.round()

39/39 [==============================] - 0s 3ms/step


In [118]:
# Print classification report
nn_report = classification_report(y_test, y_predict2)
print(nn_report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1179
           1       0.00      0.00      0.00        49

    accuracy                           0.96      1228
   macro avg       0.48      0.50      0.49      1228
weighted avg       0.92      0.96      0.94      1228



In [119]:
# Export our model to HDF5 file
# nn.save('Models/stroke_model_NN_2.h5')

**Neural Network Model 2 with RandomOverSampler:** While the overall accuracy score of the model went down, we can infer that the model is no longer classifying every point as `0` or no stroke which is an improvement on the initial model.

### **Logistic Regression Model 2**
Attempt 2 using RandomOverSampler

In [120]:
# Logistic Regression model with RandomOverSampler
# Instantiate the Logistic Regression model
ros_logistic_regression_model = LogisticRegression(max_iter=200)

# Fit the model using the resampled training data
ros_model = logistic_regression_model.fit(X_train_scaled_R, y_R)

# Make a prediction using the testing data
ros_LR_pred = logistic_regression_model.predict(X_test_scaled)

In [121]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, ros_LR_pred)

0.752895051150231

In [122]:
# Generate a confusion matrix for the model
ros_matrix = confusion_matrix(y_test, ros_LR_pred)
print(ros_matrix)

[[861 318]
 [ 11  38]]


In [123]:
# Print the classification report for the model
ros_report = classification_report(y_test, ros_LR_pred)
print(ros_report)

              precision    recall  f1-score   support

           0       0.99      0.73      0.84      1179
           1       0.11      0.78      0.19        49

    accuracy                           0.73      1228
   macro avg       0.55      0.75      0.51      1228
weighted avg       0.95      0.73      0.81      1228



**Logistic Regression Model 2 with RandomOverSampler:** Previously this model classified all data points as `0`. By oversampling our data the model is now attempting to differentiate between the `0`s and `1`s, with `76%` of the stoke values identified. correctly.

### **Neural Network Model 3**
Attempt 3 using Keras Tuner and RandomOverSampler

In [126]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 13.7 MB/s eta 0:00:00


In [127]:
import keras_tuner as kt

In [128]:
# Set value for input_dim
number_input_features = len(X_train_scaled_R[0])

# Create method for new Sequential model with hyperparameter option
def create_model (hp):
  nn_model = tf.keras.models.Sequential()

  # Activation function options
  activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  # Number of neurons in first layer
  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value=1, max_value=10, step=2),
                                     activation=activation, input_dim=number_input_features))
  
  # Number of hidden layers and neurons per layer
  for i in range(hp.Int('num_layers', 1, 3)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=1, max_value=10, step=2), activation=activation))

  nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  # Compile the model
  nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

  return nn_model


In [129]:
# Create tuner with keras_tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [130]:
# Run tuner search for best hyperparameters
tuner.search(X_train_scaled_R,y_R,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 14s]
val_accuracy: 0.7557003498077393

Best val_accuracy So Far: 0.9600977301597595
Total elapsed time: 00h 08m 15s


In [131]:
# Find most accurate model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 7,
 'num_layers': 3,
 'units_0': 7,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0,
 'units_1': 1,
 'units_2': 1}

In [132]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.6237 - accuracy: 0.9601 - 467ms/epoch - 12ms/step
Loss: 0.6237127780914307, Accuracy: 0.9600977301597595


In [133]:
# Compare predictions to actual outcomes
y_values3 = nn.predict(X_test_scaled)
y_predict3 = y_values3.round()

39/39 [==============================] - 0s 2ms/step


In [134]:
# Print classification report
nn_report = classification_report(y_test, y_predict3)
print(nn_report)

              precision    recall  f1-score   support

           0       0.97      0.80      0.88      1179
           1       0.09      0.47      0.15        49

    accuracy                           0.79      1228
   macro avg       0.53      0.64      0.52      1228
weighted avg       0.94      0.79      0.85      1228



In [ ]:
# Export our model to HDF5 file
# nn.save('Models/stroke_model_NN_3.h5')

**Neural Network Model with Keras Tuner and RandomOverSampling:** While the accuracy score shows that our model successfully classifies `96%` of the data, much of this is caused skew in our dataset. The resampling and keras tuner did improve the model's ability to identify stroke patients, but the classification report shows that this model is still less accurate than the logistic regression model.

### **K Nearest Neighbors 2**
Attempt 2 using RandomOverSampler

In [135]:
# Instantiate the model with k = 3 neighbors
model = KNeighborsClassifier(n_neighbors=3)

In [136]:
# Train the model
model.fit(X_train_scaled_R, y_R)

KNeighborsClassifier(n_neighbors=3)

In [137]:
# Create predictions
ros_KN_pred = model.predict(X_test_scaled)

In [138]:
# Print confusion matrix
confusion_matrix(ros_KN_pred,y_test)

array([[1096,   41],
       [  83,    8]])

In [139]:
# Print classification report
print(classification_report(ros_KN_pred,y_test))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95      1137
           1       0.16      0.09      0.11        91

    accuracy                           0.90      1228
   macro avg       0.55      0.53      0.53      1228
weighted avg       0.87      0.90      0.88      1228



**K Nearest Neighbors with RandomOverSampler:** The oversampling did little to improve this model's classification of true positive outcomes, while increasing the false positives.

### **Random Forest 2**
Attempt 2 using RandomOverSampler

In [140]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500)

In [141]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled_R, y_R)

In [142]:
# Making predictions using the testing data
ros_rf_pred = rf_model.predict(X_test_scaled)

In [143]:
# Print confusion matrix
confusion_matrix(ros_rf_pred,y_test)

array([[1171,   48],
       [   8,    1]])

In [144]:
# Print classification report
print(classification_report(ros_rf_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1219
           1       0.02      0.11      0.03         9

    accuracy                           0.95      1228
   macro avg       0.51      0.54      0.51      1228
weighted avg       0.99      0.95      0.97      1228



**Random Forest 2 with RandomOverSampler: ** The RandomOverSampler did little to improve this model's ability to classify positive outcomes, similiar to K Nearest Neighbors.